# Julia version of bonding models for PES !!

### Basic Optimization tests

In [88]:
using Optim

In [94]:
# standard opt:
rosenbrock(x) =  (1.0 - x[1])^2 + 99.0 * (x[2] - x[1]^2)^2
result = optimize(rosenbrock, [1.1, 2.3], BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     5.264694e-17

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 9.47e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.47e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.53e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 6.71e+01 ≰ 0.0e+00
    |g(x)|                 = 3.92e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    15
    f(x) calls:    46
    ∇f(x) calls:   46


In [104]:
# examples of "closures", a technique to fix args!
f(x::Vector, a, b) = (x[1] - a)^2 + (x[2] - b)^2

using Optim
g(x::Vector) = f(x, 3, 4)
optimize(g, [0.,0.])

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [106]:
# another closure example in Julia, anonymous function:
optimize(x -> f(x,3,4), [0.,0.]) 

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [118]:
# functional form:
function f_ratpot_2(Θ, R, Z, M)
    #=
    ansatz 1 for diatomic potential
    params:
        - Θ := training parameters, vector ()
        - R := distances, vector
        - Z := 
    =#
    # unroll coefficients
    a = Θ[1:M]
end

LoadError: syntax: expected "end" in definition of function "($ V_2)"

In [131]:
a = [0,1,2,3]
a[2:3]

2-element Vector{Int64}:
 1
 2

### 8.1 Bonding features

In [95]:
function t_R_fun(R, R_up, R_low, e)
    R2 = R.^2
    return ((R2 .- R_low^2)./(R_up^2 .- R2)).^e
end

#function s_bond_strength

t_R_fun (generic function with 3 methods)

In [97]:
R = [2.5, 3.1, 5, 1.1]
@time t_R_fun(R, 6, 1, 2)

  0.000006 seconds (2 allocations: 224 bytes)


4-element Vector{Float64}:
 0.031141868512110732
 0.1064455529835572
 4.760330578512396
 3.6435919338971527e-5